In [1]:
### 21st march ,2024

In [2]:
import pandas as pd

### merging three dataframes from three samples to make it a single dataframe

path1=('S009.xlsx')
colnames1=('chr','x1','x2')
df1=pd.read_excel(path1,names=colnames1)
df1

path2=('S010.xlsx')
colnames2=('chr','x1','x2')
df2=pd.read_excel(path2,names=colnames2)
df2

path3=('S012.xlsx')
colnames3=('chr','x1','x2')
df3=pd.read_excel(path3,names=colnames3)
df3


merged_df=pd.concat([df1,df2,df3], ignore_index=True)



/tmp/ipykernel_7302/3828319062.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Function to standardize chromosome names
def standardize_chr_name(chr_name):
    # Ensure the chromosome name is a string
    chr_name_str = str(chr_name).lower()
    
    # Remove 'chr' prefix and handle specific cases
    if chr_name_str.startswith('chr'):
        chr_name_clean = chr_name_str.replace('chr', '')
        if chr_name_clean == 'x':
            return 'X'  # Standardize to uppercase 'X' for the sex chromosome
        else:
            return chr_name_clean.upper()  # Convert back to uppercase for numerical chromosomes
    else:
        return chr_name_str.upper()  # Return the uppercase version of the original name

# Apply the standardization function to the 'chr' column
merged_df['chr'] = merged_df['chr'].apply(standardize_chr_name)

# Display the updated DataFrame
#print(merged_df)


# Display the updated DataFrame




In [4]:

import pybedtools 
from pybedtools import BedTool
merged_bed = BedTool.from_dataframe(merged_df[['chr', 'x1', 'x2']])

# merged_df is finally sorted 
sorted_merged_bed=merged_bed.sort()
df=sorted_merged_bed.to_dataframe(names=['chr','x1','x2'])
#df
#merged_df  ### un# to display

### genomic interval merging ; It would result in less rows after merging intervals

merged_sorted_merged_bed=sorted_merged_bed.merge()
#### saving the above bed file 
merged_sorted_merged_bed.saveas('merged_sorted_merged_bed.bed')
merged_sorted_merged_bed_df=merged_sorted_merged_bed.to_dataframe()

merged_sorted_merged_bed_df


#### againg sorting for safety 
sorted_merged_sorted_merged_bed=merged_sorted_merged_bed.sort()
#### making it a dtaframe to display    
sorted_merged_sorted_merged_bed_df=sorted_merged_sorted_merged_bed.to_dataframe()
#sorted_merged_sorted_merged_bed_df ## un# to display

In [5]:
####merged_sorted_merged_bed_df ###un# to display

In [6]:

##### generating unique TAD ids based on intervals x1(start) x2(end)

# Add TAD ID column with format TAD0001, TAD0002, ..., TADXXXX
import pandas as pd
# Identify unique TAD regions and assign a unique ID to each
unique_tads = sorted_merged_sorted_merged_bed_df[['chrom', 'start', 'end']].drop_duplicates().reset_index(drop=True)
unique_tads['TAD_id'] = ['TAD' + str(i).zfill(4) for i in range(1, 1 + len(unique_tads))]

# Create a mapping from each unique TAD region to its TAD ID
tad_id_mapping = unique_tads.set_index(['chrom', 'start', 'end'])['TAD_id'].to_dict()

# Map the TAD IDs back to the original DataFrame based on the TAD region
sorted_merged_sorted_merged_bed_df['TAD_id'] = sorted_merged_sorted_merged_bed_df.apply(
    lambda row: tad_id_mapping[(row['chrom'], row['start'], row['end'])], axis=1
)

# Display the DataFrame to verify the TAD_id assignment
sorted_merged_sorted_merged_bed_df
####print(sorted_merged_sorted_merged_bed_df.head(20)) 


#### sorted_merged_sorted_merged_bed_df contains TAD id but sorted_merged_sorted_merged_bed does not since sorted_merged_sorted_merged_bed_df is updated with TAD id but the bed format remained same
### need to create a a new bed file with sorted_merged_sorted_merged_bed_df

import pybedtools
from pybedtools import BedTool
bed1=BedTool.from_dataframe(sorted_merged_sorted_merged_bed_df)
#### HPV int dataset to be imported 
import pandas as pd
path=('HPVint.xlsx')
colnames=('sample ID', 'virus type', 'virus start', 'virus stop', 'Chr.', 'chr. start', 'chr. stop', 'strand1',	'strand2', '#supporting pairs',	'#split read support', 'representative break', 'nearby genes (up to four genes found within +/-500 kb from breakpoints are shown)')
df4=pd.read_excel(path,names=colnames)
df4



#### chromosome name change omitting chr

def standardize_chr_name(chr_name):
    # Ensure the chromosome name is a string
    chr_name_str = str(chr_name).lower()
    
    # Remove 'chr' prefix and handle specific cases
    if chr_name_str.startswith('chr'):
        chr_name_clean = chr_name_str.replace('chr', '')
        if chr_name_clean == 'x':
            return 'X'  # Standardize to uppercase 'X' for the sex chromosome
        else:
            return chr_name_clean.upper()  # Convert back to uppercase for numerical chromosomes
    else:
        return chr_name_str.upper()  # Return the uppercase version of the original name

# Apply the standardization function to the 'chr' column
df4['Chr.'] = df4['Chr.'].apply(standardize_chr_name)
df4

sample ID virus type  virus start  virus stop Chr.  chr. start  chr. stop  \
0     GS18001      HPV16         6031        6032   17     1104459    1104460   
1     GS18001      HPV16         5736        5737    2     9961582    9961583   
2     GS18001      HPV16         6266        6267    2     9967088    9967089   
3     GS18001      HPV16         6940        6941   21    11122389   11122390   
4     GS18001      HPV16         4523        4524    5    29687618   29687619   
..        ...        ...          ...         ...  ...         ...        ...   
217   GS21052      HPV16         2211        2212    3     2140200    2140201   
218   GS21052      HPV16         4483        4486    3     2143079    2143081   
219   GS21052      HPV16         7050        7053    3     2148095    2148098   
220   GS21052      HPV16         2457        2458    3     2148106    2148107   
221   GS21052      HPV16         1027        1047    8   129783851  129783871   

    strand1 strand2  #supporting pairs  #split read support  \
0         -       +                  2                    0   
1         +       +                  7                    9   
2         -       -                  7                    2   
3         +       +                  4                    0   
4         -       +                 24                    9   
..      ...     ...                ...                  ...   
217       -       -                111                  224   
218       -       -                  4                   33   
219       +       -                  4                   29   
220       -       +                 14                   44   
221       +       +                 12                   22   

    representative break  \
0                      Y   
1                      Y   
2                      N   
3                      Y   
4                      N   
..                   ...   
217                    Y   
218                    N   
219                    N   
220                    N   
221                    Y   

    nearby genes (up to four genes found within +/-500 kb from breakpoints are shown)  
0                               ABR,BHLHA9,TUSC5,YWHAE                                 
1                       TAF1B,GRHL1,YWHAQ,LOC101929882                                 
2                       TAF1B,GRHL1,YWHAQ,LOC101929882                                 
3                               BAGE,BAGE2,BAGE4,BAGE3                                 
4                               LOC105374704,LINC02064                                 
..                                                 ...                                 
217                                    CNTN4,CNTN4-AS2                                 
218                                    CNTN4,CNTN4-AS2                                 
219                                    CNTN4,CNTN4-AS2                                 
220                                    CNTN4,CNTN4-AS2                                 
221                      LINC00976,LINC00824,LINC00977                                 

[978 rows x 13 columns]

In [7]:
df5=df4[['Chr.','chr. start','chr. stop','sample ID','virus type']].reset_index(drop=True)
###df5


### Creating bed file from df5

import pybedtools
from pybedtools import BedTool

# Select and rename columns according to BED format expectations
df5 = df4[['Chr.', 'chr. start', 'chr. stop', 'sample ID', 'virus type']].reset_index(drop=True)
df5.columns = ['chrom', 'start', 'end', 'sample ID', 'virus type']  # Rename columns for BED format

# Creating the BED file from df5
unsorted_bed2 = BedTool.from_dataframe(df5)
unsorted_bed2=BedTool.from_dataframe(df5)
### sort bed2 
bed2=unsorted_bed2.sort()
df6=bed2.to_dataframe()
# Assuming df6 already has the columns "chrom", "start", "end" as the first three columns
# Now, adding names for the fourth and fifth columns
df6.columns = ['chrom', 'start', 'end', 'sample ID', 'viral type']

df6


chrom      start        end     sample ID viral type
0       1    1454587    1454588       GS21049      HPV16
1       1    1727131    1727132       GS18088      HPV16
2       1    2151003    2151004       GS18088      HPV16
3       1    2786393    2786394       GS18076      HPV16
4       1    9032972    9032973       GS18051      HPV16
..    ...        ...        ...           ...        ...
973     X  126094023  126094024  TCGA-CV-6961      HPV16
974     X  126094436  126094437  TCGA-CV-6961      HPV16
975     X  142664429  142664430       GS18067      HPV16
976     X  142664433  142664434       GS18067      HPV16
977     X  147554284  147554285       GS18012      HPV16

[978 rows x 5 columns]

In [8]:
###df5

In [9]:

### intersect TAD coordinates with HPV int dataset

overlap_bed=bed1.intersect(bed2, wa=True, wb=True)

overlap_bed_df=overlap_bed.to_dataframe()
overlap_bed_df.columns = ['chrom', 'start', 'end', 'TAD_id', 'Chr.', 'chr. start', 'chr. stop', 'sample ID', 'virus type']


In [10]:
overlap_bed_df.to_excel('overlap_bed_df.xlsx')

In [11]:
# Count unique samples per TAD
sample_counts_per_TAD = overlap_bed_df.groupby('TAD_id')['sample ID'].nunique()

# Identify TAD IDs with at least 3 samples
tads_of_interest_ids = sample_counts_per_TAD[sample_counts_per_TAD >= 3].index

# Filter for TADs of interest based on the identified TAD IDs
tads_of_interest = overlap_bed_df[overlap_bed_df['TAD_id'].isin(tads_of_interest_ids)]
if not tads_of_interest_ids.empty:
    print("TADs of interest based on having at least 3 samples:")
    print(tads_of_interest_ids)
else:
    print("No TAD of interest is found")

TADs of interest based on having at least 3 samples:
Index(['TAD0611', 'TAD1535', 'TAD1546', 'TAD1559', 'TAD2105', 'TAD2126'], dtype='object', name='TAD_id')


In [12]:
pybedtools.cleanup()